# preprocessing

> All code related to preprocessing our data

In [ ]:
#| default_exp preprocessing

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
import scipy.io
import numpy as np
import torch
import mne
import zarr
from mne.preprocessing import ICA

ModuleNotFoundError: No module named 'mne'

In [ ]:
#| export
def load_data(data_path, save=False):
    # Load MATLAB EEG structure
    matlab_data = scipy.io.loadmat(data_path+'/preprocessed_eeg.mat')  # Update 'your_file.mat' with your MATLAB file name

    # Extract relevant information
    fsample = float(matlab_data['preprocessed_eeg']['fsample'][0][0][0][0])  # Sampling frequency
    trial = matlab_data['preprocessed_eeg']['trial'][0][0]  # Trial data
    label = [re.findall(r"\['(.*?)'\]", str(l[0]))[0] for l in matlab_data['preprocessed_eeg']['label'][0][0]]  # Electrode labels

    # Create an MNE Info object
    info = mne.create_info(ch_names=label, sfreq=fsample, ch_types='eeg')

    # Create an MNE Raw object
    data_trial = np.stack(trial[0], axis=0)
    _, n_channels, _ = data_trial.shape
    data_trial_reshaped = data_trial.transpose(1, 0, 2).reshape(n_channels, -1)
    raw = mne.io.RawArray(data_trial_reshaped, info)
    events = [[i,0,1] for i in range(data_trial.shape[0])]
    epochs = mne.Epochs(raw, events, tmin=0, tmax=10, baseline=(0, 0))

    if save: raw.save('output_raw.fif', overwrite=True)

    return raw, events, epochs

In [ ]:
show_doc(load_data)

---

### load_data

>      load_data (data_path, save=False)

In [ ]:
#| export 
def EEG_preprocess(raw, label, get_raw=False):

    # Define the new sampling frequency and low-pass cutoff frequency
    new_sfreq = 50.0  # Hz
    lowpass_freq = 24.0  # Hz
 
    # Apply a low-pass filter to the data
    raw_filtered = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)
 
    # Resample the data
    raw_filtered = raw_filtered.copy().resample(sfreq=new_sfreq)
 
    # Fit EEG cap montage to Biowin data (127 electrodes)
    monitosed_VR1 = mne.io.read_raw_brainvision(biowin_file_path, preload=True)
    montage = monitosed_VR1.get_montage()
    electrode_info = montage.get_positions()['ch_pos']
    locs128 = np.array(list(electrode_info.values()))
    names128 = list(electrode_info.keys())
    print(len(names128))

    channels_to_remove = [item for item in label if item not in names128]
    raw_filtered.drop_channels(channels_to_remove)
    my_chan = raw_filtered.info['ch_names']
    electrode_info = {ch_name: montage.get_positions()['ch_pos'][ch_name] for ch_name in my_chan}
    locs64 = np.array(list(electrode_info.values())) 
    names64 = list(electrode_info.keys())
    missing_channels = [channel for channel in names128 if channel not in my_chan]

    if len(missing_channels)>0:

        # create an array of zeros to simulate the missing channels
        zeros_array = np.zeros((len(missing_channels), raw_filtered.n_times))
 
        # create a new RawArray object for the missing channels
        missing_info = mne.create_info(ch_names=missing_channels,
                                       sfreq=raw_filtered.info['sfreq'],
                                       ch_types='eeg')

        raw_missing = mne.io.RawArray(zeros_array, missing_info)
        raw_missing = raw_missing.filter(l_freq=None, h_freq=raw_filtered.info["lowpass"])
 
        # interpolate the missing channels
        raw_interpolated = raw_filtered.copy().add_channels([raw_missing])
        raw_interpolated = raw_interpolated.reorder_channels(names128)
 
        # Set the montage on the Raw object
        raw_interpolated.set_montage(montage)
 
        # Define bad channels 
        raw_interpolated.info['bads'] = missing_channels
        
        # Interpolate the missing channels
        raw_interpolated.interpolate_bads(reset_bads=True)

    else:
        raw_interpolated = raw_filtered.copy()
    print('interpolated: ', raw_interpolated)
 
    if get_raw:
        return raw_interpolated
 
    # Get the data from the epochs object
    data = raw_interpolated.get_data()
    # Convert to a torch tensor
    data_tensor = torch.tensor(data)
    # Print the shape of the data tensor
    print('data tensor shape: ', data_tensor.shape)
    return data_tensor

In [ ]:
show_doc(EEG_preprocess)

---

### EEG_preprocess

>      EEG_preprocess (raw, label, get_raw=False)

In [ ]:
#| export
def EEG_preprocess_eoec(raw, EC=True, get_raw=False):
    # Define epoch duration and overlap
    epoch_duration = 10  # in seconds
    overlap = 0  # in seconds

    # Define the new sampling frequency and low-pass cutoff frequency
    new_sfreq = 50.0  # Hz
    lowpass_freq = 24.0  # Hz

    # Apply a low-pass filter to the data
    raw_filtered = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)

    # Resample the data
    raw_filtered = raw_filtered.copy().resample(sfreq=new_sfreq)

    # Fit EEG cap montage to Biowin data (127 electrodes)
    montage = mne.channels.make_standard_montage('biosemi128')
    file_path = '/home/JennebauffeC/pytorchVAE/fastAI/Monitosed/Biowin/sub-001/VR_1_VRH.vhdr'
    monitosed_VR1 = mne.io.read_raw_brainvision(file_path, preload=True)
    montage = monitosed_VR1.get_montage()
    electrode_info = montage.get_positions()['ch_pos']
    names128 = list(electrode_info.keys())
    
    mapping_dict = {}
    if 'PO9' in raw_filtered.info['ch_names']:
        mapping_dict['PO9'] = 'P9'
    if 'PO10' in raw_filtered.info['ch_names']:
        mapping_dict['PO10'] = 'P10'
    raw_filtered.rename_channels(mapping_dict)

    electrode_info = raw_filtered.get_montage().get_positions()['ch_pos']
    names64 = list(electrode_info.keys())
        
    channels_to_remove = [item for item in names64 if item not in names128]
    raw_filtered.drop_channels(channels_to_remove)

    # Get the list of all channel names
    my_chan = raw_filtered.info['ch_names']

    missing_channels = [channel for channel in names128 if channel not in my_chan]

    # create an array of zeros to simulate the missing channels
    zeros_array = np.zeros((len(missing_channels), raw_filtered.n_times))

    # create a new RawArray object for the missing channels
    missing_info = mne.create_info(ch_names=missing_channels, sfreq=raw_filtered.info['sfreq'], ch_types='eeg')
    raw_missing = mne.io.RawArray(zeros_array, missing_info)
    raw_missing = raw_missing.filter(l_freq=None, h_freq=raw_filtered.info["lowpass"])
    
    # interpolate the missing channels
    raw_interpolated = raw_filtered.copy().add_channels([raw_missing])
    raw_interpolated = raw_interpolated.reorder_channels(names128)
    
    # Set the montage on the Raw object
    raw_interpolated.set_montage(montage)

    # Define bad channels 
    raw_interpolated.info['bads'] = missing_channels

    # Interpolate the missing channels
    raw_interpolated.interpolate_bads(reset_bads=True)

    if get_raw:
        return raw_interpolated

    # Convert annotations to events
    events, _ = mne.events_from_annotations(raw_interpolated)

    # Keep only the events corresponding to the majority value
    event_index = np.bincount(events[:, -1]).argmax()
    events = mne.pick_events(events, include=[event_index])

    # Extract epochs
    epochs = mne.Epochs(raw_interpolated, events, tmin=0, tmax=epoch_duration,
                        baseline=None, preload=True, event_repeated='merge')

    # Get the start times of the first epoch
    start_times = epochs.events[:, 0] / raw_interpolated.info['sfreq']

    # Define the end times of each epoch
    end_times = start_times[1:]
    
    # Iterate over the epochs and merge those that overlap
    merged_epochs = []
    i = 0
    while i < len(start_times)-2:
        start = start_times[i]
        end = end_times[i]
        j = i + 1
        while j < len(end_times) and start_times[j] - overlap <= end and end_times[j] - start <= epoch_duration:
            end = max(end, end_times[j])
            j += 1
        merged_epochs.append((start, end))
        i = j

    # Create a new list of events based on the merged epochs
    merged_events = []
    for start, end in merged_epochs:
        index = np.argmin(np.abs(events[:, 0] / raw_interpolated.info['sfreq'] - start))
        merged_events.append((int(events[index, 0]), 0, events[index, 2]))

    # Create a new Epochs object from the merged events
    merged_epochs = mne.Epochs(raw_interpolated, merged_events, tmin=0, tmax=epoch_duration,
                               baseline=None, preload=True)

    # Get the data from the epochs object
    data = merged_epochs.get_data()

    # Transpose the data array to have the channels dimension first
    data = np.transpose(data, (0, 1, 2))

    # Convert the numpy array to a torch tensor
    data_tensor = torch.from_numpy(data)
    
    # Set the corresponding label (EC vs EO)
    if EC:
        label = [0 for _ in range(data_tensor.shape[0])]
    else:
        label = [1 for _ in range(data_tensor.shape[0])]
    # Print the shape of the data tensor
    print('data tensor shape: ', data_tensor.shape)
    print('labels length: ', len(label))

    return data_tensor, label

In [ ]:
show_doc(EEG_preprocess_eoec)

---

### EEG_preprocess_eoec

>      EEG_preprocess_eoec (raw, EC=True, get_raw=False)

Nous prenons un sujet comme référence pour la construction de notre DB: 

Exemple: 

 > file_path = '/home/JennebauffeC/pytorchVAE/fastAI/Monitosed/Biowin/sub-001/VR_1_VRH.vhdr': 

In [ ]:
#| export
def get_preprocessed_EEG(files):
    x = []
    y = []

    for f in files:
        tmp_x, tmp_y = EEG_preprocess(f)
        x.append(tmp_x)
        y.extend(tmp_y)
    return x,y

In [ ]:
show_doc(get_preprocessed_EEG)

---

### get_preprocessed_EEG

>      get_preprocessed_EEG (files)

In [ ]:
#| export
def preprocess_files(file_paths, label_value):
    x_temp = []
    y_temp = []
    for file_path in tqdm(file_paths):
        # Load MATLAB EEG structure
        matlab_data = scipy.io.loadmat(file_path)
        
        # Extract relevant information
        fsample = float(matlab_data['preprocessed_eeg']['fsample'][0][0][0][0])
        trial = matlab_data['preprocessed_eeg']['trial'][0][0]  # Trial data
        label = [re.findall(r"\['(.*?)'\]", str(l[0]))[0] for l in matlab_data['preprocessed_eeg']['label'][0][0]]  # Electrode labels
        
        # Create an MNE Info object
        info = mne.create_info(ch_names=label, sfreq=fsample, ch_types='eeg')
        
        for item in tqdm(trial[0]):
            raw = mne.io.RawArray(item, info)
            processed_data = EEG_preprocess(raw, label)  # Assuming this function returns the preprocessed data
            print(processed_data.shape)
            break
            x_temp.append(processed_data)
            y_temp.append(label_value)
    
    return x_temp, y_temp

In [ ]:
show_doc(preprocess_files)

---

### preprocess_files

>      preprocess_files (file_paths, label_value)

Please respect the BIDS format

In [ ]:
#| export
def preprocess_files_eoec(eeg_dir):
    # Properly read the data
    # Get a list of all subdirectories
    subdirs = next(os.walk(eeg_dir))[1]
    # Get only the final folder names
    sub_list = [os.path.basename(os.path.normpath(subdir)) for subdir in subdirs if 'sub' in subdir]

    # Loop across all subjects
    x = []
    y = []
    for i,sub_id in enumerate(sub_list):
        clear_output(wait=True)
        print('subject ', i)
        filename = f'{sub_id}/{sub_id}_EC.set'
        filepath = os.path.join(eeg_dir, filename)
        tmp_x, tmp_y = EEG_preprocess(filepath, EC=True)
        x.append(tmp_x)
        y.extend(tmp_y)
        print('eye closed done')
        filename = f'{sub_id}/{sub_id}_EO.set'
        filepath = os.path.join(eeg_dir, filename)
        tmp_x, tmp_y = EEG_preprocess(filepath, EC=False)
        x.append(tmp_x)
        y.extend(tmp_y)

    # Convert input list to tensor
    x = torch.vstack(x)
    y_tensor = torch.tensor([[0, 1] if lab == 0 else [1, 0] for lab in y])

    return x, y_tensor

In [ ]:
show_doc(preprocess_files_eoec)

---

### preprocess_files_eoec

>      preprocess_files_eoec (eeg_dir)

In [ ]:
#| export
def create_binary_combinations(x1, x2, y1, y2, save_path_x, save_path_y):

    split_index_hyp = int(len(x1) * 0.75)
    split_index_resting = int(len(x2) * 0.75)

    x_train = x1[:split_index_hyp] + x2[:split_index_resting]
    x_val = x1[split_index_hyp:] + x2[split_index_resting:]

    y_train = y1[:split_index_hyp] + y2[:split_index_resting]
    y_val = y1[split_index_hyp:] + y2[split_index_resting:]

    x_final = x_train + x_val
    y_final = y_train + y_val

    x_final = torch.stack(x_final); x_final.shape
        
    # Save the input as Zarr file to save disk space and ease data loading
    zarr.save(save_path_x, x_final.numpy())
    # Save the labels
    torch.save(y_final, save_path_y)

In [ ]:
show_doc(create_binary_combinations)

---

### create_binary_combinations

>      create_binary_combinations (x1, x2, y1, y2, save_path_x, save_path_y)

In [ ]:
#| export
def create_epochs_from_tensors(data_path):
    # Load the Zarr file
    x_zarr = zarr.open(f'{data_path}/x.zarr', mode='r')

    # Define channel names (replace with your actual channel names)
    ch_names = [f'Channel_{i}' for i in range(127)]

    # Define channel types (replace with your actual channel types)
    ch_types = ['eeg'] * 127

    # Create an info object
    info = mne.create_info(ch_names=ch_names, sfreq=50, ch_types=ch_types)

    # Create EpochsArray
    epochs = mne.EpochsArray(x_zarr, info)

    return epochs

In [ ]:
show_doc(create_epochs_from_tensors)

---

### create_epochs_from_tensors

>      create_epochs_from_tensors (data_path)

In [ ]:
#| export
def get_ica(epochs, save_path, save=False):
    # Compute ICA components
    filt_raw = epochs.copy().filter(l_freq=1.0, h_freq=24)
    ica = ICA(n_components=42, max_iter="auto", random_state=97)
    ica.fit(filt_raw)
    ica_sources = ica.get_sources(filt_raw).get_data()
    if save: zarr.save(ica_sources, save_path)
    return ica_sources

In [ ]:
show_doc(get_ica)

---

### get_ica

>      get_ica (epochs, save_path, save=False)